In [4]:
label = 'NU_NOTA_MT'

tidy_train[label] = train_df[label]

In [5]:
df = tidy_train.select_dtypes('number')
df.fillna(-1, inplace=True)

X = df.drop('NU_NOTA_MT', axis=1)
y = df['NU_NOTA_MT']

from sklearn.model_selection import train_test_split

Xtrain, Xval, ytrain, yval = train_test_split(X,y, train_size=0.75, random_state=42)
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape 

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_absolute_error, mean_squared_error

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

k_vs_score = []
for k in range(2, Xtrain.shape[1], 2):
  #selector_model = LinearRegression(normalize=True)
  #selector_model = Ridge(alpha=0.1, normalize=True)
  selector_model = Lasso(alpha=1.,normalize=True)
  #selector_model = RandomForestRegressor(random_state=1, n_jobs=-1)
  selector = SelectFromModel(selector_model, max_features=k, threshold=-np.inf)

  selector.fit(Xtrain, ytrain)

  #Xtrain2 = np.zeros((Xtrain.shape[0], 7))
  Xtrain2 = selector.transform(Xtrain)
  #Xtrain2[:, -1] = Xtrain['START YEAR'].values

  #Xval2 = np.zeros((Xval.shape[0], 7))
  Xval2 = selector.transform(Xval)
  #Xval2[:, -1] = Xval['START YEAR'].values

  #print(Xtrain.shape, Xtrain2.shape)

  #Xtrain.columns[selector.get_support()]

  mdl = RandomForestRegressor(criterion='mse', n_estimators=1000, random_state=0, n_jobs=-1, max_depth=9)
  #mdl = XGBRegressor(objective="reg:squarederror", seed=0)
  mdl.fit(Xtrain2, ytrain)

  p = mdl.predict(Xval2)

  score = mean_absolute_error(yval, p)
  print("k = {} - MAE = {}".format(k, score))

  mask = selector.get_support()
  print(Xtrain.columns[mask])

  k_vs_score.append(score)
  #break

# k = 28 - MAE = 43.9954746831755 LR/RF
# k = 12 - MAE = 44.2187036125273 RF/RF
# k = 10 - MAE = 43.741711593358595 LASSO/RF
# k = 28 - MAE = 42.49656642909788 LASSO/RF

In [6]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor

selector_model = Lasso(alpha=1.,normalize=True)
selector = SelectFromModel(selector_model, max_features=28, threshold=-np.inf)

Xtrain = df.drop('NU_NOTA_MT', axis=1)
ytrain = df['NU_NOTA_MT']

selector.fit(Xtrain, ytrain)

Xtrain2 = selector.transform(Xtrain)

model = RandomForestRegressor(criterion='mse', n_estimators=1000, max_depth=9, n_jobs=-1, random_state=0)
model.fit(Xtrain2, ytrain);

#Xval2 = selector.transform(Xval)


#ypred = model.predict(Xval2)

#mae_rf = mean_absolute_error(yval, ypred)
#rmse_rf = np.sqrt(mean_squared_error(yval, ypred))

#print(mae_rf,rmse_rf)

In [7]:
Xtest = test_df[X.columns]
Xtest.fillna(-1, inplace=True)

Xtest2 = selector.transform(Xtest)

ypred = model.predict(Xtest2)

In [8]:
answer = pd.DataFrame({'NU_INSCRICAO':test_df.NU_INSCRICAO,
                        'TP_PRESENCA_CN':test_df.TP_PRESENCA_CN,
                        'NU_NOTA_MT':ypred})

In [9]:
for index, row in answer.iterrows():
    if row['TP_PRESENCA_CN'] == 0:
        answer.loc[index,'NU_NOTA_MT'] = np.nan
answer['NU_NOTA_MT'] = answer['NU_NOTA_MT'].apply(lambda x: 0 if x < 250 else x)

In [10]:
answer.drop('TP_PRESENCA_CN', axis=1, inplace=True)
answer.to_csv('answer.csv', index=False)


In [11]:
! codenation submit -c enem-2

Versão: 1.0.14

Usando arquivo de configuração: /home/vcwild/.codenation.yml

Executando testes...

{"score": 93.75923111050471}
Preparando code review... 100% |████████████████████████████████████████|  [9s:0s]
Códigos submetidos com sucesso! 

Sua nota é: 93.759231

Parabéns! Você superou este desafio!

O que fazer agora?
- Você pode continuar sua jornada escolhendo um novo desafio em https://www.codenation.com.br
- Você pode ajudar outros desenvolvedores revisando códigos ou respondendo dúvidas no forum do desafio
- Lembre-se que ensinar é uma ótima forma de ganhar reconhecimento e gera um bom karma ;)


from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor


fs = []
for seed in range(1000):

  np.random.seed(seed)
  k = np.random.randint(2, 20, 1)[0]
  selected = np.random.choice(Xtrain.columns, k, replace=False)
  #print(selected)
  
  Xtrain2 = Xtrain[selected]
  Xval2 = Xval[selected]
  
  #mdl = RandomForestRegressor(n_estimators=1000, random_state=seed, n_jobs=-1)
  mdl = XGBRegressor(objective="reg:squarederror", seed=0)
  mdl.fit(Xtrain2, ytrain)

  p = mdl.predict(Xval2)

  score = mean_absolute_error(yval, p)
  if score < 46:
    print("seed = {} - k = {} - MAE = {}".format(seed, k, score))
    fs.append(p)
